In [1]:
# Load the modules
import numpy as np
import tensorflow as tf
import math
import pickle
import matplotlib.pyplot as plt

# Keras imports
from tqdm import tqdm
from keras.layers import Conv2D, Flatten
from keras.layers.core import Dropout
from keras.layers.pooling import MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense, Input, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
# read training data from train.p
# read testing data from test.p

TRAIN_FILE = 'camera.pickle'

# open file in reading and binary mode
with open(TRAIN_FILE, 'rb') as f:
    pickle_data = pickle.load(f)
    X_train = pickle_data['train_dataset']
    y_train = pickle_data['train_labels']
    X_validation = pickle_data['valid_dataset']
    y_validation = pickle_data['valid_labels']
    X_test = pickle_data['test_dataset']
    y_test = pickle_data['test_labels']
    del pickle_data  # Free up memory

print('Training data shape', X_train.shape)
print('Validation data shape', X_validation.shape)
print('Testing data shape', X_test.shape)


Training data shape (16272, 18, 80, 1)
Validation data shape (5425, 18, 80, 1)
Testing data shape (2411, 18, 80, 1)


In [3]:
#normalize the data between range -0.5 to 0.5
def normalize_between_plus_min_05(image_data):
    image_data = image_data.astype('float32')
    image_data = image_data / 255 - 0.5
    return image_data

X_train = normalize_between_plus_min_05(X_train)
X_validation = normalize_between_plus_min_05(X_validation)
X_test = normalize_between_plus_min_05(X_test)
print('Shape of image:', X_train.shape[1:])


Shape of image: (18, 80, 1)


In [4]:
# Set the parameters and print out the summary of the model
np.random.seed(1337)  # for reproducibility

BATCH_SIZE = 64 # The lower the better
N_CLASSES = 1 # The output is a single digit: a steering angle
EPOCHS = 10 # The higher the better


In [5]:
#input shape
input_shape = X_train.shape[1:]


# number of convolutional filters to use
nb_filters1 = 16
nb_filters2 = 8
nb_filters3 = 4
nb_filters4 = 2

# size of pooling area for max pooling
pool_size = (2, 2)

# convolution kernel size
kernel_size = (3, 3)

# Initiating the model
model = Sequential()

# Starting with the convolutional layer
# The first layer will turn 1 channel into 16 channels
model.add(Conv2D(nb_filters1, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
# Applying ReLU
model.add(Activation('relu'))
# The second conv layer will convert 16 channels into 8 channels
model.add(Conv2D(nb_filters2, kernel_size[0], kernel_size[1]))
# Applying ReLU
model.add(Activation('relu'))
# The second conv layer will convert 8 channels into 4 channels
model.add(Conv2D(nb_filters3, kernel_size[0], kernel_size[1]))
# Applying ReLU
model.add(Activation('relu'))
# The second conv layer will convert 4 channels into 2 channels
model.add(Conv2D(nb_filters4, kernel_size[0], kernel_size[1]))
# Applying ReLU
model.add(Activation('relu'))
# Apply Max Pooling for each 2 x 2 pixels
model.add(MaxPooling2D(pool_size=pool_size))
# Apply dropout of 25%
model.add(Dropout(0.25))

# Flatten the matrix. The input has size of 360
model.add(Flatten())
# Input 360 Output 16
model.add(Dense(16))
# Applying ReLU
model.add(Activation('relu'))
# Input 16 Output 16
model.add(Dense(16))
# Applying ReLU
model.add(Activation('relu'))
# Input 16 Output 16
model.add(Dense(16))
# Applying ReLU
model.add(Activation('relu'))
# Apply dropout of 50%
model.add(Dropout(0.5))
# Input 16 Output 1
model.add(Dense(N_CLASSES))

model.summary()


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 16, 78, 16)    160         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 16, 78, 16)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 14, 76, 8)     1160        activation_1[0][0]               
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 14, 76, 8)     0           convolution2d_2[0][0]            
___________________________________________________________________________________________

In [6]:
model.compile(loss='mean_squared_error',
              optimizer=Adam(),
              metrics=['accuracy'])

### Model training
history = model.fit(X_train, y_train,
                    batch_size=BATCH_SIZE, nb_epoch=EPOCHS,
                    verbose=1, validation_data=(X_validation, y_validation))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])


Train on 16272 samples, validate on 5425 samples
Epoch 1/10
16272/16272 [==============================] - 20s - loss: 0.0146 - acc: 0.5423 - val_loss: 0.0114 - val_acc: 0.5469

In [7]:
import json

# Save model as json file
json_string = model.to_json()

with open('model.json', 'w') as outfile:
    json.dump(json_string, outfile)
    # save weights
    model.save_weights('./model.h5')
    print("Saved")

Saved
